In [13]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os

username = "user"  
password = "grad5100user"  
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"  
port = 3306
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)


Question 1 and 2

In [17]:
chefdetails_query = """
SELECT season, seasonNumber, name, placement, gender
FROM chefdetails
WHERE season NOT LIKE '%%Masters%%' AND season != 'Canada 6';
"""
chefdetails = pd.read_sql(chefdetails_query, engine)

judges_query = """
SELECT season, seasonNumber, episode, challengeType, guestJudge, competedOnTC
FROM judges
WHERE season NOT LIKE '%%Masters%%' AND season != 'Canada 6';
"""
judges = pd.read_sql(judges_query, engine)

In [16]:
chefdetails

,season,seasonNumber,name,placement,gender
0,All Stars: New York,8,Richard Blais,1,Male
1,All Stars: New York,8,Mike Isabella,2,Male
2,All Stars: New York,8,Antonia Lofaso,3,Female
3,All Stars: New York,8,Tiffany Derry,4,Female
4,All Stars: New York,8,Carla Hall,5,Female
...,...,...,...,...,...
317,World All Stars,20,May Phattanant Thongthong,14,Female
318,World All Stars,20,Begona Rodrigo,12,Female
319,World All Stars,20,Gabriel Rodriguez,2,Male
320,World All Stars,20,Amar Santana,6,Male


In [18]:
judges

,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None
...,...,...,...,...,...,...
628,World All Stars,20,12,Quickfire,Sam Bompas,None
629,World All Stars,20,12,Elimination,Jeremy Chan,None
630,World All Stars,20,13,Quickfire,Greg Marchand,None
631,World All Stars,20,13,Elimination,Greg Marchand,None


Question 3

In [20]:
seasons_query = "SELECT DISTINCT season, seasonNumber FROM chefdetails WHERE season NOT LIKE '%%Masters%%' AND season != 'Canada 6';"
seasons = pd.read_sql(seasons_query, engine)
seasons

,season,seasonNumber
0,All Stars: New York,8
1,All-Stars L.A.,17
2,Boston,12
3,California,13
4,Charleston,14
5,Chicago,4
6,D.C.,7
7,Colorado,15
8,Houston,19
9,Kentucky,16


Question 4

In [23]:
gender_count = chefdetails['gender'].value_counts()
gender_count

Male      170
Female    152
Name: gender, dtype: int64

In [24]:
winners_gender_count = chefdetails[chefdetails['placement'] == 1]['gender'].value_counts()
winners_gender_count

Male      14
Female     6
Name: gender, dtype: int64

In [25]:
top3_gender_count = chefdetails[chefdetails['placement'].isin([1, 2, 3])]['gender'].value_counts()
top3_gender_count

Male      36
Female    25
Name: gender, dtype: int64

In [28]:
top3_female_count = chefdetails[(chefdetails['placement'].isin([1, 2, 3])) & (chefdetails['gender'] == 'Female')].shape[0]
female_winners_count = chefdetails[(chefdetails['placement'] == 1) & (chefdetails['gender'] == 'Female')].shape[0]
top3_female_count

25

In [29]:
female_winners_count

6

It does seem unusual that there are only 6 female winners considering that they were in the top 3 25 times. I do believe that it is less likely for a woman who places top 3 to win considering that 19 out of 25 times they lost when being in the top 3.

In [30]:
winners = chefdetails[chefdetails['placement'] == 1]
winners_judges = pd.merge(winners, judges, how='left', left_on='name', right_on='guestJudge')
winners_judges['served_as_judge'] = winners_judges['guestJudge'].apply(lambda x: 'Yes' if pd.notna(x) else 'No')


In [32]:
winners_judges

,season_x,seasonNumber_x,name,placement,gender,season_y,seasonNumber_y,episode,challengeType,guestJudge,competedOnTC,served_as_judge
0,All Stars: New York,8,Richard Blais,1,Male,Boston,12.0,1.0,Sudden Death Quickfire,Richard Blais,Yes,Yes
1,All Stars: New York,8,Richard Blais,1,Male,Boston,12.0,1.0,Elimination,Richard Blais,Yes,Yes
2,All Stars: New York,8,Richard Blais,1,Male,Boston,12.0,8.0,Elimination,Richard Blais,Yes,Yes
3,All Stars: New York,8,Richard Blais,1,Male,Colorado,15.0,6.0,Quickfire,Richard Blais,Yes,Yes
4,All Stars: New York,8,Richard Blais,1,Male,Kentucky,16.0,3.0,Quickfire,Richard Blais,Yes,Yes
5,All Stars: New York,8,Richard Blais,1,Male,Portland,18.0,2.0,Quickfire,Richard Blais,Yes,Yes
6,All Stars: New York,8,Richard Blais,1,Male,Portland,18.0,10.0,Quickfire,Richard Blais,Yes,Yes
7,All-Stars L.A.,17,Melissa King,1,Female,Houston,19.0,4.0,Elimination,Melissa King,Yes,Yes
8,All-Stars L.A.,17,Melissa King,1,Female,Houston,19.0,10.0,Elimination,Melissa King,Yes,Yes
9,All-Stars L.A.,17,Melissa King,1,Female,Portland,18.0,2.0,Quickfire,Melissa King,Yes,Yes


Question 5

In [34]:
non_winners_judges = pd.merge(chefdetails[chefdetails['placement'] != 1], judges, how='inner', left_on='name', right_on='guestJudge')
non_winners_judges = non_winners_judges[['name']].drop_duplicates()
non_winners_judges

,name
0,Mike Isabella
2,Antonia Lofaso
4,Tiffany Derry
8,Dale Talde
10,Spike Mendelsohn
12,Stephen Asprinio
14,Bryan Voltaggio
22,Gregory Gourdet
24,Karen Akunowicz
26,Lee Anne Wong
